In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,58,2024-02-28,-1,118.596491,99.756140,0.563140,10.738596,22.198246,0.200965,118.887719,...,11.350000,25.206897,0.186052,115.524138,0,2,2,3,7,2024
1,59,2024-02-28,-2,116.086207,98.001724,0.555569,11.405172,25.634483,0.209052,118.298276,...,11.247458,23.396610,0.178780,121.820339,0,2,2,3,19,2024
2,57,2024-02-28,1,114.446429,97.283929,0.558625,11.569643,24.430357,0.187411,117.271429,...,10.654386,24.454386,0.187702,115.492982,0,2,2,3,6,2024
3,59,2024-02-28,-2,106.500000,97.856897,0.516966,12.525862,21.756897,0.181707,108.477586,...,12.807018,23.142105,0.220789,116.871930,0,2,2,2,15,2024
4,58,2024-02-28,-1,118.491228,99.159649,0.565912,11.282456,23.407018,0.174649,118.359649,...,10.712069,26.227586,0.178931,119.301724,0,2,2,2,26,2024
5,60,2024-02-28,-1,116.915254,100.283051,0.560441,11.801695,19.547458,0.214542,115.713559,...,11.289286,24.850000,0.218357,120.867857,0,2,2,3,14,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-28,DAL,TOR,1.645161,2.300000,1
1,2024-02-28,NOP,IND,2.950000,1.420168,0
2,2024-02-28,CLE,CHI,1.487805,2.700000,0
3,2024-02-28,MEM,MIN,6.250000,1.133333,0
4,2024-02-28,SAC,DEN,3.400000,1.335570,0
5,2024-02-28,LAL,LAC,2.360000,1.617284,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-28,DAL,TOR,1.645161,2.300000,1
1,2024-02-28,NOP,IND,2.950000,1.420168,0
2,2024-02-28,CLE,CHI,1.487805,2.700000,0
3,2024-02-28,MEM,MIN,6.250000,1.133333,0
4,2024-02-28,SAC,DEN,3.400000,1.335570,0
5,2024-02-28,LAL,LAC,2.360000,1.617284,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-28,DAL,TOR,1.645161,2.300000,1
1,2024-02-28,NOP,IND,2.950000,1.420168,0
2,2024-02-28,CLE,CHI,1.487805,2.700000,0
3,2024-02-28,MEM,MIN,6.250000,1.133333,0
4,2024-02-28,SAC,DEN,3.400000,1.335570,0
5,2024-02-28,LAL,LAC,2.360000,1.617284,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-28,DAL,TOR,1.645161,2.300000,0
1,2024-02-28,NOP,IND,2.950000,1.420168,0
2,2024-02-28,CLE,CHI,1.487805,2.700000,0
3,2024-02-28,MEM,MIN,6.250000,1.133333,0
4,2024-02-28,SAC,DEN,3.400000,1.335570,0
5,2024-02-28,LAL,LAC,2.360000,1.617284,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-28,DAL,TOR,1.645161,2.300000,1.0
1,2024-02-28,NOP,IND,2.950000,1.420168,0.0
2,2024-02-28,CLE,CHI,1.487805,2.700000,1.0
3,2024-02-28,MEM,MIN,6.250000,1.133333,0.0
4,2024-02-28,SAC,DEN,3.400000,1.335570,0.0
5,2024-02-28,LAL,LAC,2.360000,1.617284,0.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-28,DAL,TOR,1.645161,2.300000,1.0
1,2024-02-28,NOP,IND,2.950000,1.420168,0.0
2,2024-02-28,CLE,CHI,1.487805,2.700000,1.0
3,2024-02-28,MEM,MIN,6.250000,1.133333,0.0
4,2024-02-28,SAC,DEN,3.400000,1.335570,0.0
5,2024-02-28,LAL,LAC,2.360000,1.617284,0.0
